In [ ]:
import os
import xml.etree.ElementTree as ET

image_dir = "traffic_sign_data/images"
annotation_dir = "traffic_sign_data/Annotations" 
label_dir = "traffic_sign_data/labels"

classes = ['stop', 'no_parking', 'speed_limit', 'turn_left', 'turn_right']  

os.makedirs(label_dir, exist_ok=True)

def convert(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[2]) / 2.0 * dw
    y = (box[1] + box[3]) / 2.0 * dh
    w = (box[2] - box[0]) * dw
    h = (box[3] - box[1]) * dh
    return (x, y, w, h)

for xml_file in os.listdir(annotation_dir):
    if not xml_file.endswith('.xml'):
        continue

    tree = ET.parse(os.path.join(annotation_dir, xml_file))
    root = tree.getroot()
    
    img_width = int(root.find("size/width").text)
    img_height = int(root.find("size/height").text)

    yolo_lines = []
    for obj in root.findall("object"):
        cls_name = obj.find("name").text
        if cls_name not in classes:
            continue
        cls_id = classes.index(cls_name)

        xml_box = obj.find("bndbox")
        b = (
            int(xml_box.find("xmin").text),
            int(xml_box.find("ymin").text),
            int(xml_box.find("xmax").text),
            int(xml_box.find("ymax").text),
        )
        bb = convert((img_width, img_height), b)
        yolo_lines.append(f"{cls_id} {' '.join(f'{a:.6f}' for a in bb)}")

    txt_filename = xml_file.replace(".xml", ".txt")
    with open(os.path.join(label_dir, txt_filename), "w") as out_file:
        out_file.write("\n".join(yolo_lines))
